# TEI Conference 2016 #

## Etape 2 : Normalisation des documents ##

Dans cette étape, nous allons d'une part nettoyer les abstracts de leur encodage pour produire des textes bruts, et d'autre part nous allons créer un fichier csv qui réunira toutes les informations et les métadonnées de chaque abstract. Celui-ci fonctionne comme un index.

## Les packages ## 

Le package OS permet de naviguer dans le cache.

Le package csv permet de créer le document csv contenant les métadonnées des abstracts.

La librairie beautifulsoup permet de naviguer dans l'arbre XML de chaque abstract et de sélectionner les informations que nous recherchons : les métadonnées d'abord, le texte de l'abstract ensuite.

La librairie TQDM permet d'afficher les logs de chargement, ce qui est un peu plus joli.

Le package " re " permet d'utiliser des regex, que nous utiliserons pour nettoyer les balises.

In [1]:
import os
import csv
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

## Travail sur le texte ## 

Nous allons maintenant travailler le texte.

Tout d'abord, la cellule ci-dessous permet de créer le dossier cache accueillant les textes bruts, et d'initialiser les variables d'entrée et de sortie des fichiers.

In [2]:
Path = "./cache2016/cacheXML/"
Path_output = "./cache2016/cacheTXT/"
filelist = os.listdir(Path)
if not os.path.exists("./cache2016/cacheTXT"):
    os.makedirs("./cache2016/cacheTXT")

La cellule suivante nous permet de créer une fonction qui supprime tout ce qui commence par un chevron ouvrant et qui se termine par un chevron fermant.

La seule solution fiable que j'ai trouvé pour nettoyer le texte de ses balises est d'utiliser une regex.
L'avantage de cette solution est qu'elle permet de faire fi des erreurs d'encodage et des erreurs de lecture par le parseur xml.
L'inconvénient est qu'il supprime les balises d'exemple : parfois, un abstract mentionne une balise non pas en tant que balise encodant le texte, mais comme texte lui-même. La balise disparaît. 

Il a été choisi d'utiliser la regex ci-dessous car le trop grand nombre d'erreurs relevées par le parseur XML (lxml de etree) rendait son utilisation trop compliquée pour les gains potentiels.

In [3]:
def nettoyage_balises(texte):
    p = re.compile(r'<.*?>')
    return p.sub('', texte)

Dans le programme ci-dessous, j'ouvre chaque abstract un par un et je stocke en variable à chaque fois les éléments que je souhaite récupérer (titre, auteur(es), université de rattachement). 
Je stocke en variable le corps de texte de l'abstract, je le nettoie de ses balises et j'ouvre un fichier texte brut dans lequel j'écris le titre de l'abstract (cela fait à mes yeux partie du texte, et pour une étude de texte le titre est un morceau de choix) et le texte.
J'attribue à ce fichier texte brut un titre normalisé (Papers + année + numéro unique) que je stocke en variable.

Finalement, j'écris dans le csv le nom des auteurs, le titre original de l'abstract, l'université de rattachement, les mots-clés donnés dans les métadonnées et le titre normalisé attribué dans le cache.

In [4]:
i = 0 #compteur permettant de numéroter le nom des abstracts sous une forme normalisée
year = 2016 #variable permettant de nommer les abstracts sous une forme normalisée.
with open('./cache2016/TEI2016.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for abstract in filelist:
        with open(Path + abstract, 'r', encoding="UTF-8") as y:
            value = y.read()
            soup = BeautifulSoup(value, 'xml')
            print("Récupération des autorités...")
            for contributeur in tqdm(soup.find_all('author')): #Je récupère tous les noeuds "author"
                for elem in soup.name: #Je vais récupérer ici les éléments de prénom et de nom de chaque auteur
                    liste_noms_balise = soup.find_all(['forename', 'surname'])
                    noms_a_rassembler = [] #j'instancie une liste dans lequel je vais ranger chaque élément de nom ou de prénom de tous les auteurs d'un seul abstract
                    for nom in liste_noms_balise: #ici, je vais normaliser en les transformant en str et en retirant les balises manuellement.
                        nom = nettoyage_balises(str(nom))
                        noms_a_rassembler.append(nom)
                for elem in soup.titleStmt:
                    titre = soup.title.contents #grâce à la méthode .contents, je récupère directement une str, sous condition qu'il n'y ait pas de balises imbriquées (comme pour les noms)
                    if len(titre) > 1: #permet de récupérer un contenu si l'auteur a rajouté des balises dans le titre, ce qui perd le parseur
                        titre = titre[0]
                    titre = "".join(titre) #permet de nettoyer les espaces parasites, avec les deux lignes suivantes
                    titre = titre.split()
                    titre = " ".join(titre)
                    titre = [titre]
                for elem in soup.publicationStmt: #Ici je récupere directement une str grâce à la méthode .contents.
                    organisation = soup.publisher.contents
                    orga = organisation[0].split() #Pour éliminer les sauts de lignes, tab et autres espaces gênants, la solution la plus simple fut d'enchaîner un .split et un .join
                    orga = ' '.join(orga)
                    orga = [orga]
                print("Récupération des mots-clés...")
                for elem in tqdm(soup.keywords):
                    liste_motcles = soup.find_all('term') #je récupère tous les mots clés dans une liste
                    retypage_liste = [str(i) for i in liste_motcles] #je transforme le type des éléments de la liste en str chacun
                    motcles = str(" | ".join(retypage_liste)) #je transforme la liste en str
                    motcles = motcles.replace("<term>", "") #je nettoie la str
                    motcles = motcles.replace("</term>", "")
                    motcles = [motcles] #je passe cette grande str en une liste avec 1 seul élément
            abstract = soup("body") #ici, j'enregistre le contenu de l'abstract, et je ne garde que le premier élément, car les autres éléments capturés sont trop compliqués à traiter, et trop peu intéressants.
            abstract = abstract[0]
            titre_dans_cache = "paper-{0}-{1}.txt".format(year, i)
            with open('./cache2016/cacheTXT/%s.txt' % titre_dans_cache, 'w', encoding='UTF-8') as docTXT:#stockage des textes en cache
                text = nettoyage_balises(str(abstract)) #permet de retirer toutes les balises
                list_mots = text.split()#permet, avec la ligne suivante, de supprimer tous les espaces parasites laissés par la suppression des balises
                texte = " ".join(list_mots)
                for elem in titre:#c'est important de noter le titre
                    docTXT.write(elem)
                docTXT.write("\n")
                docTXT.write(texte)
            titre_dans_cache = titre_dans_cache.replace(".txt", "")
            noms_rassembles = ' '.join(noms_a_rassembler) #tous les noms saisis plus haut, chacun étant un élément d'une grande liste, sont rassemblés dans une str, qui est ensuite transformée en une liste d'un seul élément
            noms = [noms_rassembles]
            titre_dans_cache = [titre_dans_cache]
            masterList = noms + titre + orga + motcles + titre_dans_cache #j'assemble chacune des listes composées d'un seul élément en une nested liste. Logiquement, cela ressemble à : [[str], [str], [str], [str]]
            writer.writerow(masterList)
        i += 1
        

100%|██████████| 3/3 [00:00<00:00, 2565.85it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Récupération des autorités...
Récupération des mots-clés...
Récupération des mots-clés...
Récupération des autorités...



100%|██████████| 3/3 [00:00<00:00, 626.02it/s]

100%|██████████| 3/3 [00:00<00:00, 1537.31it/s]

Récupération des mots-clés...
Récupération des mots-clés...
Récupération des mots-clés...




100%|██████████| 3/3 [00:00<00:00, 748.72it/s]

 43%|████▎     | 6/14 [00:00<00:00, 10.98it/s]

...

Récupération des mots-clés...
Récupération des mots-clés...
Récupération des mots-clés...




100%|██████████| 3/3 [00:00<00:00, 2328.44it/s]

100%|██████████| 3/3 [00:00<00:00, 1137.59it/s]

100%|██████████| 2/2 [00:00<00:00, 21.89it/s]

Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des mots-clés...



100%|██████████| 3/3 [00:00<00:00, 2576.35it/s]

100%|██████████| 3/3 [00:00<00:00, 2542.00it/s]

100%|██████████| 2/2 [00:00<00:00, 24.70it/s]

Récupération des autorités...
Récupération des mots-clés...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des mots-clés...
